# Construccion RDC01- patrimonio EF

In [1]:
from pathlib import Path
import pandas as pd
import numpy as np
import bcchapi

# ✅ Mostrar todas las columnas al imprimir DataFrames
pd.set_option('display.max_columns', None)
#fecha a reportar
# === FECHA DE CONSULTA ===
fecha_reporte = pd.to_datetime("2025-11-21")


## conexion banco central 

In [2]:
import sys
from pathlib import Path

ruta_BC = (
    Path.home()
    / "EF Securitizadora"
    / "Operaciones - Documentos"
    / "Documentos"
    / "Operaciones"
    / "REDEC"
)

if str(ruta_BC) not in sys.path:
    sys.path.append(str(ruta_BC))

from BANCO_CENTRAL.banco_central import obtener_valores_bc

print("Módulo importado OK.")

valor_usd, valor_uf, df_bc = obtener_valores_bc(fecha_reporte)

print("USD:", valor_usd)
print("UF:", valor_uf)

Módulo importado OK.
USD: 929.0
UF: 39643.59


In [3]:
# Definir la ruta base
# 1️⃣ Ruta base y nombre del archivo
ruta_seguros = (
    Path.home()
    / 'EF Securitizadora'
    / 'Operaciones - Documentos'
    / 'Documentos'
    / 'Operaciones'
    / 'REDEC'
    / 'EF'
    / 'EF'
    / 'seguro'
)

seguros = ruta_seguros / 'Seguros.xlsx'


# Leer el archivo Excel
df_seguros = pd.read_excel(seguros)

df_seguros["N°Operación"] = df_seguros["N°Operación"].astype("Int64")

df_seguros["valor_Garantia_inmobiliaria"] = df_seguros['Tasacion Vivienda']
df_seguros["valor_Garantia_mobiliaria"] = 0

# 4 Eliminar guion del RUT
df_seguros['rut_id'] = df_seguros['RUT'].astype(str).str.replace('-', '', regex=False)

# 5 Crear columna TIPO_PERSONA sin dígito verificador
df_seguros['TIPO_PERSONA'] = df_seguros['rut_id'].str[:-1]

# 6 Convertir a número (ignorando errores por seguridad)
df_seguros['TIPO_PERSONA'] = pd.to_numeric(df_seguros['TIPO_PERSONA'], errors='coerce')

# 7 Clasificar: 1 = Persona Natural / 2 = Persona Jurídica
df_seguros['TIPO_PERSONA'] = df_seguros['TIPO_PERSONA'].apply(
    lambda x: 1 if x < 50000000 else 2
)

df_garantia = df_seguros

In [4]:
df_garantia[df_garantia['N° Operación CMF']=='202410CLH0658']

,RUT,N°Operación,N° Operación CMF,Fecha Otorgamiento,Fecha de Vencimiento,Tasacion Vivienda,Valor Vivienda,Moneda,valor_Garantia_inmobiliaria,valor_Garantia_mobiliaria,rut_id,TIPO_PERSONA
1,88002059,100010003638,202410CLH0658,2008-01-04,2038-02-01,1346.0,1345.0,UF,1346.0,0,88002059,1


In [5]:
df_garantia.columns

Index(['RUT', 'N°Operación', 'N° Operación CMF', 'Fecha Otorgamiento',
       'Fecha de Vencimiento', 'Tasacion Vivienda', 'Valor Vivienda', 'Moneda',
       'valor_Garantia_inmobiliaria', 'valor_Garantia_mobiliaria', 'rut_id',
       'TIPO_PERSONA'],
      dtype='object')

In [6]:
df_garantia.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40 entries, 0 to 39
Data columns (total 12 columns):
 #   Column                       Non-Null Count  Dtype         
---  ------                       --------------  -----         
 0   RUT                          40 non-null     object        
 1   N°Operación                  40 non-null     Int64         
 2   N° Operación CMF             40 non-null     object        
 3   Fecha Otorgamiento           40 non-null     object        
 4   Fecha de Vencimiento         40 non-null     datetime64[ns]
 5   Tasacion Vivienda            40 non-null     float64       
 6   Valor Vivienda               40 non-null     float64       
 7   Moneda                       40 non-null     object        
 8   valor_Garantia_inmobiliaria  40 non-null     float64       
 9   valor_Garantia_mobiliaria    40 non-null     int64         
 10  rut_id                       40 non-null     object        
 11  TIPO_PERSONA                 40 non-null     in

In [7]:
# 2️⃣ Ruta base y nombre del archivo
ruta_control_amort = (
    Path.home()
    / 'EF Securitizadora'
    / 'Administración y Finanzas - Documentos'
    / 'Documentos'
    / 'ADMT Y FINANZAS'
    / 'PAULA TORO'
    / 'Automatizaciones'
    / 'PROPIEDADES'
    / 'CONTROL_AMORTIZACIONES'
    / '2025-11'
    / 'Formato antiguo'
)

archivo_control_amort = ruta_control_amort / 'Control_amort_11-2025 HIPOTECARIA.xlsx'
#Cargar solo los nombres de las hojas

with pd.ExcelFile(archivo_control_amort) as xls:
    print(xls.sheet_names)

df_cartera = pd.read_excel(archivo_control_amort, sheet_name='AMORTIZACION')
df_cartera.head()
#df_amort.to_excel("AMORTIZACION_20251114.xlsx", index=False)





#se actualizara la cartera por eso usare por ahora esta ruta:

# 1️⃣ Ruta base y nombre del archivo
#ruta_amort = (
#    Path.home()
#    / 'EF Securitizadora'
#    / 'Operaciones - Documentos'
#    / 'Documentos'
#    / 'Operaciones'
#    / 'REDEC'
#    / 'EF'
#    / 'EF'
#    / 'amortizacion'
#)

#amortizacion = ruta_amort / "AMORTIZACION_20251114.xlsx"
# Leer el archivo Excel
#df_cartera = pd.read_excel(amortizacion)



['Fecha', 'UF_TABLA', 'REMESAS DIARIAS', 'Consol_remesas', 'Respaldo_consolidado', 'CARTOLAS', 'AMORTIZACION', 'AMORT SIST']


,OPERACIÓN,PLAZO,FECHA,CUOTA/CUPON,INTERES,AMORTIZACIÓN,SALDO INSOLUTO,LLAVE,LLAVE 2,UF F_PAGO,INT. PENALES,$ BANCO,F. ABO BANC,ESTADO,ABONO_TOTA,PAT,Columna1
0,100010023635,1,2013-09-10,0.000000,0.000000,0.000000,433.043651,92013100010023635,1000100236351,0.0,0.0,0.0,NaT,PAGADO,NaN,PS005,1000100236351
1,100010023635,2,2013-10-10,4.475188,3.969697,0.505492,432.538160,102013100010023635,1000100236352,0.0,0.0,0.0,NaT,PAGADO,NaN,PS005,1000100236352
2,100010023635,3,2013-11-10,4.475188,3.964976,0.510212,432.027948,112013100010023635,1000100236353,0.0,0.0,0.0,NaT,PAGADO,NaN,PS005,1000100236353
3,100010023635,4,2013-12-10,4.475188,3.960256,0.514932,431.513016,122013100010023635,1000100236354,0.0,0.0,0.0,NaT,PAGADO,NaN,PS005,1000100236354
4,100010023635,5,2014-01-10,4.475188,3.955536,0.519652,430.993364,12014100010023635,1000100236355,0.0,0.0,0.0,NaT,PAGADO,NaN,PS005,1000100236355


In [8]:
df_cartera.dtypes

OPERACIÓN                  int64
PLAZO                      int64
FECHA             datetime64[ns]
CUOTA/CUPON              float64
INTERES                  float64
AMORTIZACIÓN             float64
SALDO INSOLUTO           float64
LLAVE                      int64
LLAVE 2                    int64
UF F_PAGO                float64
INT. PENALES             float64
$ BANCO                  float64
F. ABO BANC       datetime64[ns]
ESTADO                    object
ABONO_TOTA               float64
PAT                       object
Columna1                   int64
dtype: object

In [9]:

# Interés diario
df_cartera["interes_diario"] = df_cartera["INTERES"] / 30

# Días de devengo
df_cartera["dias_devengo"] = (df_cartera["FECHA"] - fecha_reporte).dt.days.clip(lower=0)

# Inicializar columnas como float
df_cartera["saldo_insoluto_actual"] = 0.0
df_cartera["cuota_morosa"] = 0.0

# Saldo insoluto de la última cuota pagada antes del reporte
for op, grupo in df_cartera.groupby("OPERACIÓN"):
    pagadas = grupo[(grupo["ESTADO"] == "PAGADO") & (grupo["FECHA"] <= fecha_reporte)]
    if not pagadas.empty:
        ultima = pagadas.sort_values("FECHA").iloc[-1]
        idx = ultima.name
        df_cartera.at[idx, "saldo_insoluto_actual"] = ultima["SALDO INSOLUTO"]

# Cuotas morosas (fecha vencida + estado pendiente)
condicion = (df_cartera["FECHA"] < fecha_reporte) & (df_cartera["ESTADO"] == "PENDIENTE")
df_cartera.loc[condicion, "cuota_morosa"] = df_cartera.loc[condicion, "CUOTA/CUPON"]

In [10]:

# 1. Obtener la primera cuota por vencer después de la fecha de reporte
primeras_vencidas = (
    df_cartera[df_cartera["FECHA"] > fecha_reporte]
    .sort_values(["OPERACIÓN", "FECHA"])
    .groupby("OPERACIÓN", as_index=False)
    .first()[["OPERACIÓN", "interes_diario", "dias_devengo"]]
)

# 2. Obtener el último saldo_insoluto_actual válido por operación
saldo_insoluto_actual = (
    df_cartera[df_cartera["saldo_insoluto_actual"] > 0]
    .groupby("OPERACIÓN", as_index=False)["saldo_insoluto_actual"]
    .max()
)

# 3. Sumar todas las cuotas morosas por operación
suma_cuotas_vencidas = (
    df_cartera[df_cartera["cuota_morosa"] > 0]
    .groupby("OPERACIÓN", as_index=False)["cuota_morosa"]
    .sum()
    .rename(columns={"cuota_morosa": "suma_cuotas_vencida"})
)

# 4. Combinar toda la información
df_monto_actual = saldo_insoluto_actual.merge(primeras_vencidas, on="OPERACIÓN", how="left")
df_monto_actual = df_monto_actual.merge(suma_cuotas_vencidas, on="OPERACIÓN", how="left")

# 5. Reemplazar NaN por cero en suma_cuotas_vencida
df_monto_actual["suma_cuotas_vencida"] = df_monto_actual["suma_cuotas_vencida"].fillna(0)

df_monto_actual['monto_actual_con_devengo'] = df_monto_actual['saldo_insoluto_actual'] + df_monto_actual['suma_cuotas_vencida'] + df_monto_actual['interes_diario'] * df_monto_actual['dias_devengo']

In [11]:
df_monto_actual


,OPERACIÓN,saldo_insoluto_actual,interes_diario,dias_devengo,suma_cuotas_vencida,monto_actual_con_devengo
0,100010000340,116.780667,0.003546,19,107.830720,224.678769
1,100010000357,13.596174,0.003044,19,0.000000,13.654006
2,100010000453,24.694083,0.005800,19,0.000000,24.804276
3,100010000482,20.888207,0.005416,19,0.000000,20.991120
4,100010000506,28.639648,0.007721,19,0.000000,28.786338
5,100010000510,28.941864,0.006022,19,5.942866,34.999152
6,100010002063,6.976100,0.001523,19,0.904600,7.909643
7,100010002437,155.435723,0.025162,19,46.647660,202.561467
8,100010002648,113.105274,0.025291,19,0.000000,113.585805
9,100010002862,61.320015,0.016517,19,0.000000,61.633835


In [12]:
#df_monto_actual[df_monto_actual['OPERACIÓN']==	100010000340]

In [13]:
#df_cartera[df_cartera['OPERACIÓN'] == 100010023635].iloc[140:150]

In [14]:
#df_cartera[df_cartera['OPERACIÓN'] ==	100010000340].iloc[225:240]


In [15]:
#df_cartera

In [16]:
df_garantia.dtypes

RUT                                    object
N°Operación                             Int64
N° Operación CMF                       object
Fecha Otorgamiento                     object
Fecha de Vencimiento           datetime64[ns]
Tasacion Vivienda                     float64
Valor Vivienda                        float64
Moneda                                 object
valor_Garantia_inmobiliaria           float64
valor_Garantia_mobiliaria               int64
rut_id                                 object
TIPO_PERSONA                            int64
dtype: object

In [17]:
df_garantia[df_garantia['N°Operación']==100010023635]

,RUT,N°Operación,N° Operación CMF,Fecha Otorgamiento,Fecha de Vencimiento,Tasacion Vivienda,Valor Vivienda,Moneda,valor_Garantia_inmobiliaria,valor_Garantia_mobiliaria,rut_id,TIPO_PERSONA
0,135833479,100010023635,202410CLH0654,2013-08-22,2033-08-22,681.0,613.0,UF,681.0,0,135833479,1


In [18]:
def construccion_mora(df_filtrado, fecha_reporte, valor_usd, valor_uf):
    """
    Construcción de mora según estándar CMF (RDC01):
    - MONTO AL DÍA = solo cuotas con estado 'AL DÍA' y vencimiento > fecha de reporte.
    - TRAMOS 1–9 = valor_cuota completo de cuotas morosas.
    """

    fecha_reporte = pd.to_datetime(fecha_reporte)
    resultados = []

    operaciones = df_filtrado["OPERACIÓN"].unique()

    for op in operaciones:
        dn = df_filtrado[df_filtrado["OPERACIÓN"] == op].copy()

        # Calcular días de mora
        dn["dias_mora"] = (fecha_reporte - dn["FECHA"]).dt.days
        dn["dias_mora"] = dn["dias_mora"].clip(lower=0)

        # === MONTO AL DÍA ===
        al_dia = dn[
            (dn["FECHA"] > fecha_reporte) &
            (dn["ESTADO"] == 'PENDIENTE')
        ]
        monto_al_dia = al_dia["CUOTA/CUPON"].sum()

        # === CUOTAS MOROSAS ===
        vencidas = dn[
            (dn["FECHA"] < fecha_reporte) &
            (dn["ESTADO"] == 'PENDIENTE')
        ]

        # === TRAMOS DE MORA CMF ===
        tramos_def = {
            "Tramo_1": (1, 30),
            "Tramo_2": (30, 60),
            "Tramo_3": (60, 90),
            "Tramo_4": (90, 180),
            "Tramo_5": (180, 365),
            "Tramo_6": (365, 730),
            "Tramo_7": (730, 1095),
            "Tramo_8": (1095, 1460),
            "Tramo_9": (1460, float("inf")),
        }

        mora_tramos = {}
        for tramo, (ini, fin) in tramos_def.items():
            filtro = (vencidas["dias_mora"] >= ini) & (vencidas["dias_mora"] < fin)
            mora_tramos[tramo] = vencidas.loc[filtro, "CUOTA/CUPON"].sum()

        # === ÚLTIMA CUOTA VIGENTE ===
        cuotas_vigentes = dn[dn["FECHA"] <= fecha_reporte].sort_values("FECHA", ascending=False)
        if not cuotas_vigentes.empty:
            ultima_cuota_valor = cuotas_vigentes.iloc[0]["CUOTA/CUPON"]
        else:
            futura = dn[dn["FECHA"] > fecha_reporte]
            ultima_cuota_valor = futura["CUOTA/CUPON"].iloc[0] if not futura.empty else 0

        resultados.append({
            "operacion": op,
            "valor_cuota": ultima_cuota_valor,
            "dias_mora": vencidas["dias_mora"].max() if not vencidas.empty else 0,
            "Monto_al_dia": monto_al_dia,
            **mora_tramos
        })

    mora = pd.DataFrame(resultados)
    # === CONVERSIÓN A CLP ===
    columnas_conversion = [
        "valor_cuota", "Monto_al_dia",
        "Tramo_1", "Tramo_2", "Tramo_3", "Tramo_4",
        "Tramo_5", "Tramo_6", "Tramo_7", "Tramo_8", "Tramo_9","valor_cuota_cmf","Leasing_Monto_original"
    ]

    

    mora["valor_cuota_cmf"] = mora["valor_cuota"]
    # MONTO ORIGINAL DE LA OPERACIÓN para leasing
    mora["Leasing_Monto_original"] = (
        mora["Monto_al_dia"] +
        mora["Tramo_1"] + mora["Tramo_2"] + mora["Tramo_3"] +
        mora["Tramo_4"] + mora["Tramo_5"] + mora["Tramo_6"] +
        mora["Tramo_7"] + mora["Tramo_8"] + mora["Tramo_9"]
    )

    #mora.loc[mora["moneda"] == "CLP", columnas_conversion] *= 1
    #mora.loc[mora["moneda"] == "USD", columnas_conversion] *= valor_usd
    #mora.loc[mora["moneda"] == "UF", columnas_conversion] *= valor_uf

    return mora

In [19]:
mora = construccion_mora(df_cartera, fecha_reporte, valor_usd, valor_uf)
mora.head(16)

,operacion,valor_cuota,dias_mora,Monto_al_dia,Tramo_1,Tramo_2,Tramo_3,Tramo_4,Tramo_5,Tramo_6,Tramo_7,Tramo_8,Tramo_9,valor_cuota_cmf,Leasing_Monto_original
0,100010023635,4.475188,11,416.023844,4.475188,0.000000,0.000000,0.000000,0.00000,0.0,0.0,0.0,0.0,4.475188,420.499032
1,100010003638,9.419008,20,1374.294970,9.419008,0.000000,0.000000,0.000000,0.00000,0.0,0.0,0.0,0.0,9.419008,1383.713978
2,100010000357,13.562532,0,13.687488,0.000000,0.000000,0.000000,0.000000,0.00000,0.0,0.0,0.0,0.0,13.562532,13.687488
3,100010002437,11.661915,103,116.726595,11.661915,11.661915,11.661915,11.661915,0.00000,0.0,0.0,0.0,0.0,11.661915,163.374255
4,100010000482,5.345070,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.0,0.0,0.0,0.0,5.345070,0.000000
5,100010018412,7.217324,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.0,0.0,0.0,0.0,7.217324,0.000000
6,100010005160,5.507690,0,181.923960,0.000000,0.000000,0.000000,0.000000,0.00000,0.0,0.0,0.0,0.0,5.507690,181.923960
7,100010029218,6.476091,0,615.375954,0.000000,0.000000,0.000000,0.000000,0.00000,0.0,0.0,0.0,0.0,6.476091,615.375954
8,100010000227,6.569864,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.0,0.0,0.0,0.0,6.569864,0.000000
9,100010000207,11.953974,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.0,0.0,0.0,0.0,11.953974,0.000000


In [20]:
df_monto_actual = df_monto_actual.dropna().copy()

df_monto_actual.head()

,OPERACIÓN,saldo_insoluto_actual,interes_diario,dias_devengo,suma_cuotas_vencida,monto_actual_con_devengo
0,100010000340,116.780667,0.003546,19,107.83072,224.678769
1,100010000357,13.596174,0.003044,19,0.00000,13.654006
2,100010000453,24.694083,0.005800,19,0.00000,24.804276
3,100010000482,20.888207,0.005416,19,0.00000,20.991120
4,100010000506,28.639648,0.007721,19,0.00000,28.786338


In [21]:
dk=df_cartera[df_cartera['OPERACIÓN']==	100010023635	]
dk.iloc[140:150]

,OPERACIÓN,PLAZO,FECHA,CUOTA/CUPON,INTERES,AMORTIZACIÓN,SALDO INSOLUTO,LLAVE,LLAVE 2,UF F_PAGO,INT. PENALES,$ BANCO,F. ABO BANC,ESTADO,ABONO_TOTA,PAT,Columna1,interes_diario,dias_devengo,saldo_insoluto_actual,cuota_morosa
140,100010023635,141,2025-05-10,4.475188,2.677646,1.797542,290.312225,52025100010023635,100010023635141,39186.92,0.016701,176023.294892,NaT,PAGADO,2915643.0,PS005,100010023635141,0.089255,0,0.000000,0.000000
141,100010023635,142,2025-06-10,4.475188,2.661340,1.813848,288.498377,62025100010023635,100010023635142,39267.07,0.018182,176441.474326,NaT,PAGADO,2505078.0,PS005,100010023635142,0.088711,0,0.000000,0.000000
142,100010023635,143,2025-07-10,4.475188,2.644605,1.830583,286.667794,72025100010023635,100010023635143,39163.82,0.020192,176056.253152,2025-08-06,PAGADO,4440380.0,PS005,100010023635143,0.088153,0,0.000000,0.000000
143,100010023635,144,2025-08-10,4.475188,2.627870,1.847319,284.820475,82025100010023635,100010023635144,39360.32,0.015985,176774.006455,2025-09-03,PAGADO,2247221.0,PS005,100010023635144,0.087596,0,0.000000,0.000000
144,100010023635,145,2025-09-10,4.475188,2.610705,1.864483,282.955992,92025100010023635,100010023635145,39485.65,0.016827,177370.132085,2025-10-01,PAGADO,1208416.0,PS005,100010023635145,0.087023,0,0.000000,0.000000
145,100010023635,146,2025-10-10,4.475188,2.593970,1.881218,281.074774,102025100010023635,100010023635146,39592.57,0.016827,177850.418329,2025-11-05,PAGADO,4276194.0,PS005,100010023635146,0.086466,0,281.074774,0.000000
146,100010023635,147,2025-11-10,4.475188,2.576376,1.898812,279.175962,112025100010023635,100010023635147,0.00,0.000000,0.000000,NaT,PENDIENTE,NaN,PS005,100010023635147,0.085879,0,0.000000,4.475188
147,100010023635,148,2025-12-10,4.475188,2.559212,1.915976,277.259986,122025100010023635,100010023635148,0.00,0.000000,0.000000,NaT,PENDIENTE,NaN,PS005,100010023635148,0.085307,19,0.000000,0.000000
148,100010023635,149,2026-01-10,4.475188,2.541619,1.933570,275.326416,12026100010023635,100010023635149,0.00,0.000000,0.000000,NaT,PENDIENTE,NaN,PS005,100010023635149,0.084721,50,0.000000,0.000000
149,100010023635,150,2026-02-10,4.475188,2.524025,1.951163,273.375253,22026100010023635,100010023635150,0.00,0.000000,0.000000,NaT,PENDIENTE,NaN,PS005,100010023635150,0.084134,81,0.000000,0.000000


In [22]:
mora.dtypes

operacion                   int64
valor_cuota               float64
dias_mora                   int64
Monto_al_dia              float64
Tramo_1                   float64
Tramo_2                   float64
Tramo_3                   float64
Tramo_4                   float64
Tramo_5                   float64
Tramo_6                   float64
Tramo_7                   float64
Tramo_8                   float64
Tramo_9                   float64
valor_cuota_cmf           float64
Leasing_Monto_original    float64
dtype: object

In [23]:
#dp = df_cartera[df_cartera['OPERACIÓN'] == 100010000340]
#dp.tail(10)

# nos quedamos con la PRIMERA cuota NO pagada (por fecha)
#primera_impaga = (
#    dp
#    .loc[~df_cartera["pagada"]]
#    .sort_values("FECHA")
#    .iloc[0]
#)

#fecha_base = primera_impaga["FECHA"]
#interes_base = primera_impaga["INTERES"]
#saldo_insoluto_base = primera_impaga["SALDO INSOLUTO"]
#print(saldo_insoluto_base)
#print(interes_base)
#print(saldo_insoluto_base)

#dias_devengo = (fecha_reporte - fecha_base).days
#dias_devengo = max(dias_devengo, 0)

#interes_devengado = interes_base * (dias_devengo / 30)

#MONTO_ACTUAL_OPERACION = saldo_insoluto_base + interes_devengado
#dp.tail(10)

In [24]:
# 1. Asegurar tipos consistentes
mora["operacion"] = mora["operacion"].astype(int)
df_garantia["N°Operación"] = df_garantia["N°Operación"].astype(int)
df_monto_actual["OPERACIÓN"] = df_monto_actual["OPERACIÓN"].astype(int)

# 2. Realizar merge en dos pasos: mora ↔ garantía ↔ monto_actual
df_merge = (
    mora.merge(df_garantia, left_on="operacion", right_on="N°Operación", how="left")
        .merge(df_monto_actual[["OPERACIÓN", "monto_actual_con_devengo"]], left_on="operacion", right_on="OPERACIÓN", how="left")
        .fillna({"valor_Garantia_inmobiliaria": 0, "valor_Garantia_mobiliaria": 0})
)

# 3. Construir DataFrame final prueba_RDC01
prueba_RDC01 = pd.DataFrame({
    "RUT": df_merge["rut_id"],
    "TIPO_PERSONA": df_merge["TIPO_PERSONA"],
    "CODIGO_OPERACION": df_merge["operacion"],
    "CODIGO_OPERACION_CMF": df_merge["N° Operación CMF"],
    "OPERACION_TITULO_III": 4,
    "TIPO_DEUDOR": 1,
    "TIPO_CREDITO": 32,
    "FECHA_OTORGAMIENTO": df_merge["Fecha Otorgamiento"],
    "CARGA_FINANCIERA": df_merge["valor_cuota_cmf"]*valor_uf,
    "FECHA_EXTINCION_OPERACION": df_merge["Fecha de Vencimiento"],
    "VALOR_GARANTIA_INMOBILIARIA": df_merge["valor_Garantia_inmobiliaria"]*valor_uf,
    "VALOR_GARANTIA_MOBILIARIA": df_merge["valor_Garantia_mobiliaria"]*valor_uf,
    "VALOR_GARANTIA_FINANCIERA": 0,
    "VALOR_GARANTIA_AVAL_FIANZA": 0,
    "MONTO_ORIGINAL_OPERACION": df_merge["Leasing_Monto_original"]*valor_uf,
    "MONTO_ACTUAL_OPERACION": df_merge["monto_actual_con_devengo"]*valor_uf,
    "MONTO_AL_DIA": df_merge["Monto_al_dia"]*valor_uf,
    "MONTO_MORA_1_TRAMO": df_merge["Tramo_1"]*valor_uf,
    "MONTO_MORA_2_TRAMO": df_merge["Tramo_2"]*valor_uf,
    "MONTO_MORA_3_TRAMO": df_merge["Tramo_3"]*valor_uf,
    "MONTO_MORA_4_TRAMO": df_merge["Tramo_4"]*valor_uf,
    "MONTO_MORA_5_TRAMO": df_merge["Tramo_5"]*valor_uf,
    "MONTO_MORA_6_TRAMO": df_merge["Tramo_6"]*valor_uf,
    "MONTO_MORA_7_TRAMO": df_merge["Tramo_7"]*valor_uf,
    "MONTO_MORA_8_TRAMO": df_merge["Tramo_8"]*valor_uf,
    "MONTO_MORA_9_TRAMO": df_merge["Tramo_9"]*valor_uf,
    "MORA_ACTUAL": df_merge["dias_mora"],
    "DEUDA_RENEGOCIADA": 2,
    "DEUDA_ACELERADA": 2
})

# Opcional: eliminar nulos
df_RDC01 = prueba_RDC01.dropna().copy()

In [25]:
df_RDC01[df_RDC01['CODIGO_OPERACION']==100010023635]

,RUT,TIPO_PERSONA,CODIGO_OPERACION,CODIGO_OPERACION_CMF,OPERACION_TITULO_III,TIPO_DEUDOR,TIPO_CREDITO,FECHA_OTORGAMIENTO,CARGA_FINANCIERA,FECHA_EXTINCION_OPERACION,VALOR_GARANTIA_INMOBILIARIA,VALOR_GARANTIA_MOBILIARIA,VALOR_GARANTIA_FINANCIERA,VALOR_GARANTIA_AVAL_FIANZA,MONTO_ORIGINAL_OPERACION,MONTO_ACTUAL_OPERACION,MONTO_AL_DIA,MONTO_MORA_1_TRAMO,MONTO_MORA_2_TRAMO,MONTO_MORA_3_TRAMO,MONTO_MORA_4_TRAMO,MONTO_MORA_5_TRAMO,MONTO_MORA_6_TRAMO,MONTO_MORA_7_TRAMO,MONTO_MORA_8_TRAMO,MONTO_MORA_9_TRAMO,MORA_ACTUAL,DEUDA_RENEGOCIADA,DEUDA_ACELERADA
0,135833479,1.0,100010023635,202410CLH0654,4,1,32,2013-08-22,177412.518245,2033-08-22,26997284.79,0.0,0,0,1.667009e+07,1.138448e+07,1.649268e+07,177412.518245,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,11,2,2


In [26]:
dn = df_garantia[df_garantia['N°Operación']== 100010003638]
dn

,RUT,N°Operación,N° Operación CMF,Fecha Otorgamiento,Fecha de Vencimiento,Tasacion Vivienda,Valor Vivienda,Moneda,valor_Garantia_inmobiliaria,valor_Garantia_mobiliaria,rut_id,TIPO_PERSONA
1,88002059,100010003638,202410CLH0658,2008-01-04,2038-02-01,1346.0,1345.0,UF,1346.0,0,88002059,1


In [27]:
df_monto_actual[df_monto_actual['OPERACIÓN']==100010003638	]

,OPERACIÓN,saldo_insoluto_actual,interes_diario,dias_devengo,suma_cuotas_vencida,monto_actual_con_devengo
19,100010003638,878.264676,0.195585,10,9.419008,889.639537


In [28]:
df_RDC01[df_RDC01['CODIGO_OPERACION'] == 100010023635]

,RUT,TIPO_PERSONA,CODIGO_OPERACION,CODIGO_OPERACION_CMF,OPERACION_TITULO_III,TIPO_DEUDOR,TIPO_CREDITO,FECHA_OTORGAMIENTO,CARGA_FINANCIERA,FECHA_EXTINCION_OPERACION,VALOR_GARANTIA_INMOBILIARIA,VALOR_GARANTIA_MOBILIARIA,VALOR_GARANTIA_FINANCIERA,VALOR_GARANTIA_AVAL_FIANZA,MONTO_ORIGINAL_OPERACION,MONTO_ACTUAL_OPERACION,MONTO_AL_DIA,MONTO_MORA_1_TRAMO,MONTO_MORA_2_TRAMO,MONTO_MORA_3_TRAMO,MONTO_MORA_4_TRAMO,MONTO_MORA_5_TRAMO,MONTO_MORA_6_TRAMO,MONTO_MORA_7_TRAMO,MONTO_MORA_8_TRAMO,MONTO_MORA_9_TRAMO,MORA_ACTUAL,DEUDA_RENEGOCIADA,DEUDA_ACELERADA
0,135833479,1.0,100010023635,202410CLH0654,4,1,32,2013-08-22,177412.518245,2033-08-22,26997284.79,0.0,0,0,1.667009e+07,1.138448e+07,1.649268e+07,177412.518245,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,11,2,2


In [29]:
A = df_cartera[df_cartera['OPERACIÓN'] == 100010011126].iloc[173:174]
A


,OPERACIÓN,PLAZO,FECHA,CUOTA/CUPON,INTERES,AMORTIZACIÓN,SALDO INSOLUTO,LLAVE,LLAVE 2,UF F_PAGO,INT. PENALES,$ BANCO,F. ABO BANC,ESTADO,ABONO_TOTA,PAT,Columna1,interes_diario,dias_devengo,saldo_insoluto_actual,cuota_morosa
6617,100010011126,174,2025-11-01,6.141156,2.714313,3.426843,306.797148,112025100010011126,100010011126174,0.0,0.0,0.0,NaT,PENDIENTE,NaN,PS006,100010011126174,0.090477,0,0.0,6.141156


In [30]:
A = df_cartera[df_cartera['OPERACIÓN'] == 100010011126].iloc[173:174]
A


,OPERACIÓN,PLAZO,FECHA,CUOTA/CUPON,INTERES,AMORTIZACIÓN,SALDO INSOLUTO,LLAVE,LLAVE 2,UF F_PAGO,INT. PENALES,$ BANCO,F. ABO BANC,ESTADO,ABONO_TOTA,PAT,Columna1,interes_diario,dias_devengo,saldo_insoluto_actual,cuota_morosa
6617,100010011126,174,2025-11-01,6.141156,2.714313,3.426843,306.797148,112025100010011126,100010011126174,0.0,0.0,0.0,NaT,PENDIENTE,NaN,PS006,100010011126174,0.090477,0,0.0,6.141156


In [31]:
B = df_cartera[df_cartera['OPERACIÓN'] == 100010003638].iloc[201:202]
B

,OPERACIÓN,PLAZO,FECHA,CUOTA/CUPON,INTERES,AMORTIZACIÓN,SALDO INSOLUTO,LLAVE,LLAVE 2,UF F_PAGO,INT. PENALES,$ BANCO,F. ABO BANC,ESTADO,ABONO_TOTA,PAT,Columna1,interes_diario,dias_devengo,saldo_insoluto_actual,cuota_morosa
441,100010003638,202,2024-11-01,9.419008,6.163938,3.25507,915.547238,112024100010003638,100010003638202,38108.63,0.0,358945.490839,2024-11-27,PAGADO,2951012.0,PS005,100010003638202,0.205465,0,0.0,0.0


In [32]:
C = df_cartera[df_cartera['OPERACIÓN'] == 100010008557].iloc[182:183]
C



,OPERACIÓN,PLAZO,FECHA,CUOTA/CUPON,INTERES,AMORTIZACIÓN,SALDO INSOLUTO,LLAVE,LLAVE 2,UF F_PAGO,INT. PENALES,$ BANCO,F. ABO BANC,ESTADO,ABONO_TOTA,PAT,Columna1,interes_diario,dias_devengo,saldo_insoluto_actual,cuota_morosa
10706,100010008557,183,2025-11-01,6.739216,2.960896,3.778321,292.46866,112025100010008557,100010008557183,39643.59,0.010132,267568.384879,NaT,PAGADO,1108972.0,PS006,100010008557183,0.098697,0,292.46866,0.0


In [33]:
D = df_cartera[df_cartera['OPERACIÓN'] == 100010003664].iloc[209:210]
D


,OPERACIÓN,PLAZO,FECHA,CUOTA/CUPON,INTERES,AMORTIZACIÓN,SALDO INSOLUTO,LLAVE,LLAVE 2,UF F_PAGO,INT. PENALES,$ BANCO,F. ABO BANC,ESTADO,ABONO_TOTA,PAT,Columna1,interes_diario,dias_devengo,saldo_insoluto_actual,cuota_morosa
5273,100010003664,210,2025-11-01,11.496828,2.15243,9.344398,311.560508,112025100010003664,100010003664210,0.0,0.0,0.0,NaT,PENDIENTE,NaN,PS005,100010003664210,0.071748,0,0.0,11.496828


In [34]:
df_RDC01 = df_RDC01.drop(columns=['CODIGO_OPERACION'])
df_RDC01 = df_RDC01.rename(columns={"CODIGO_OPERACION_CMF": "CODIGO_OPERACION"})
df_RDC01.columns

Index(['RUT', 'TIPO_PERSONA', 'CODIGO_OPERACION', 'OPERACION_TITULO_III',
       'TIPO_DEUDOR', 'TIPO_CREDITO', 'FECHA_OTORGAMIENTO', 'CARGA_FINANCIERA',
       'FECHA_EXTINCION_OPERACION', 'VALOR_GARANTIA_INMOBILIARIA',
       'VALOR_GARANTIA_MOBILIARIA', 'VALOR_GARANTIA_FINANCIERA',
       'VALOR_GARANTIA_AVAL_FIANZA', 'MONTO_ORIGINAL_OPERACION',
       'MONTO_ACTUAL_OPERACION', 'MONTO_AL_DIA', 'MONTO_MORA_1_TRAMO',
       'MONTO_MORA_2_TRAMO', 'MONTO_MORA_3_TRAMO', 'MONTO_MORA_4_TRAMO',
       'MONTO_MORA_5_TRAMO', 'MONTO_MORA_6_TRAMO', 'MONTO_MORA_7_TRAMO',
       'MONTO_MORA_8_TRAMO', 'MONTO_MORA_9_TRAMO', 'MORA_ACTUAL',
       'DEUDA_RENEGOCIADA', 'DEUDA_ACELERADA'],
      dtype='object')

In [35]:
df_RDC01

,RUT,TIPO_PERSONA,CODIGO_OPERACION,OPERACION_TITULO_III,TIPO_DEUDOR,TIPO_CREDITO,FECHA_OTORGAMIENTO,CARGA_FINANCIERA,FECHA_EXTINCION_OPERACION,VALOR_GARANTIA_INMOBILIARIA,VALOR_GARANTIA_MOBILIARIA,VALOR_GARANTIA_FINANCIERA,VALOR_GARANTIA_AVAL_FIANZA,MONTO_ORIGINAL_OPERACION,MONTO_ACTUAL_OPERACION,MONTO_AL_DIA,MONTO_MORA_1_TRAMO,MONTO_MORA_2_TRAMO,MONTO_MORA_3_TRAMO,MONTO_MORA_4_TRAMO,MONTO_MORA_5_TRAMO,MONTO_MORA_6_TRAMO,MONTO_MORA_7_TRAMO,MONTO_MORA_8_TRAMO,MONTO_MORA_9_TRAMO,MORA_ACTUAL,DEUDA_RENEGOCIADA,DEUDA_ACELERADA
0,135833479,1.0,202410CLH0654,4,1,32,2013-08-22,177412.518245,2033-08-22,26997284.79,0.0,0,0,1.667009e+07,1.138448e+07,1.649268e+07,177412.518245,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.0,0.0,0.0,11,2,2
1,88002059,1.0,202410CLH0658,4,1,32,2008-01-04,373403.291359,2038-02-01,53360272.14,0.0,0,0,5.485539e+07,3.526851e+07,5.448199e+07,373403.291359,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.0,0.0,0.0,20,2,2
2,74154492,1.0,202410CLH0653,4,1,32,2005-12-16,537667.457970,2025-12-16,71001669.69,0.0,0,0,5.426212e+05,5.412938e+05,5.426212e+05,0.000000,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.0,0.0,0.0,0,2,2
3,106923485,1.0,202410CLH0655,4,1,32,2006-09-08,462320.176875,2026-09-08,57839997.81,0.0,0,0,6.476742e+06,8.030264e+06,4.627461e+06,462320.176875,462320.176875,462320.176875,4.623202e+05,0.000000e+00,0.000000e+00,0.0,0.0,0.0,103,2,2
6,126348045,1.0,202410CLH0633,4,1,32,2008-08-29,218344.604207,2028-08-29,28265879.67,0.0,0,0,7.212119e+06,6.393285e+06,7.212119e+06,0.000000,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.0,0.0,0.0,0,2,2
7,69683428,1.0,202410CLH0650,4,1,32,2013-10-30,256735.496407,2033-10-30,45471197.73,0.0,0,0,2.439571e+07,1.718610e+07,2.439571e+07,0.000000,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.0,0.0,0.0,0,2,2
12,6449189K,1.0,202410CLH0656,4,1,32,2006-09-13,464686.304544,2026-09-14,63390100.41,0.0,0,0,4.651144e+06,4.502949e+06,4.651144e+06,0.000000,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.0,0.0,0.0,0,2,2
14,74531296,1.0,202410CLH0642,4,1,32,2006-02-15,326920.468499,2026-02-15,40713966.93,0.0,0,0,9.928696e+05,9.833305e+05,9.928696e+05,0.000000,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.0,0.0,0.0,0,2,2
15,68837707,1.0,202410CLH0652,4,1,32,2005-12-22,534349.606636,2025-12-22,67711251.72,0.0,0,0,4.814048e+06,8.907073e+06,5.392514e+05,534349.606636,534349.606636,534349.606636,1.603049e+06,1.068699e+06,0.000000e+00,0.0,0.0,0.0,225,2,2
18,7487519K,1.0,202410CLH0644,4,1,32,2006-03-30,291082.028842,2026-03-30,32309525.85,0.0,0,0,1.158349e+06,1.141194e+06,1.158349e+06,0.000000,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.0,0.0,0.0,0,2,2


In [ ]:
fecha_str = fecha_reporte.strftime("%Y%m%d")  # → 20251114

# ============================
# 📁 RUTA BASE
# ============================
ruta_excel_dir = (
    Path.home()
    / "EF Securitizadora"
    / "Operaciones - Documentos"
    / "Documentos"
    / "Operaciones"
    / "REDEC"
    / "EF"
    / "EF"
    / "archivo_xlsx"
)

ruta_excel_dir.mkdir(parents=True, exist_ok=True)

# ============================
# 📄 Nombre final del archivo
# ============================
nombre_excel = f"RDC01_EF_{fecha_str}.xlsx"
ruta_salida_excel = ruta_excel_dir / nombre_excel

# ============================
# 💾 Guardar Excel
# ============================
df_RDC01.to_excel(ruta_salida_excel, index=False)

print("✅ Excel guardado correctamente en:")
print(ruta_salida_excel)

✅ Excel guardado correctamente en:
C:\Users\MarceloVidal\EF Securitizadora\Operaciones - Documentos\Documentos\Operaciones\REDEC\EF\EF\archivo_xlsx\RDC01_EF_20251121.xlsx


: 

In [ ]:

#1.RUT
#R: df_seguros['RUT']
#2.TIPO PERSONA
#R: 1 si es persona natural y 2 si es persona juridica
#3.CÓDIGO OPERACIÓN
#R: df_seguro[''N°Operación''] y df_amort['OPERACIÓN']
#4.OPERACIÓN TÍTULO III
#R: 4


#5.TIPO DE DEUDOR
#R: tipo de deudor, directo === 1 


#6.TIPO DE CRÉDITO
#R: 32


#7.FECHA DE OTORGAMIENTO
#R: df_seguros [''Fecha Otorgamiento'']

#8.CARGA FINANCIERA: valor cuota
#R: 

#9.FECHA DE EXTINCIÓN DE LA OPERACIÓN
#R: df_seguros[Fecha de Vencimiento]




#10. VALOR DE LA GARANTÍA REAL INMOBILIARIA clp
#R: df_seguro['Tasacion Vivienda']

#11.VALOR DE LA GARANTÍA REAL MOBILIARIA clp
#R:0

#12. VALOR DE LA GARANTÍA FINANCIERA
#R:0
#13.VALOR DE LA GARANTÍA AVAL O FIANZA
#R:0

#14.MONTO ORIGINAL DE LA OPERACIÓN clp
#R: cuotas por vencer + si tiene mora

#15.MONTO ACTUAL DE LA OPERACIÓN clp
#R: saldo insoluto + interes

#16.MONTO AL DÍA clp
#R: cuotas por vencer 

#17.MONTO MORA 1er TRAMO clp
#R: mora['Tramo_1']
#18.MONTO MORA 2do TRAMO clp
#R:mora['Tramo_2']
#19.MONTO MORA 3er TRAMO clp
#R:mora['Tramo_3']
#20.MONTO MORA 4to TRAMO clp
#R:mora['Tramo_4']
#21.MONTO MORA 5to TRAMO clp
#R:mora['Tramo_5']
#22. MONTO MORA 6to TRAMO clp
#R:mora['Tramo_6']
#23.MONTO MORA 7mo TRAMO clp
#R:mora['Tramo_7']
#24.MONTO MORA 8vo TRAMO clp
#R:mora['Tramo_8']
#25.MONTO MORA 9no TRAMO clp
#R:mora['Tramo_9']
#26.MORA ACTUAL ->dias
#R:mora['dias_mora']


#27.DEUDA RENEGOCIADA
#R:2
#28.DEUDA ACELERADA
#R:2
